In [15]:
!sudo apt-get install -y python3-xmltodict
import xmltodict

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-xmltodict is already the newest version (0.12.0-1).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.


In [16]:
import logging
import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

In [65]:
def parse_into_dict(xmlfile):
    import xmltodict
    with open(xmlfile) as ifp:
        doc = xmltodict.parse(ifp.read())
        #print(doc)
        #print("%%")
        return doc

In [66]:
document = parse_into_dict('orders.xml')
print(document)

OrderedDict([('Root', OrderedDict([('@xmlns', 'http://www.adventure-works.com'), ('Orders', OrderedDict([('Order', [OrderedDict([('CustomerID', 'GREAL'), ('EmployeeID', '6'), ('OrderDate', '1997-05-06T00:00:00'), ('RequiredDate', '1997-05-20T00:00:00'), ('ShipInfo', OrderedDict([('@ShippedDate', '1997-05-09T00:00:00'), ('ShipVia', '2'), ('Freight', '3.35'), ('ShipName', 'Great Lakes Food Market'), ('ShipAddress', '2732 Baker Blvd.'), ('ShipCity', 'Eugene'), ('ShipRegion', 'OR'), ('ShipPostalCode', '97403'), ('ShipCountry', 'USA')]))]), OrderedDict([('CustomerID', 'GREAL'), ('EmployeeID', '8'), ('OrderDate', '1997-07-04T00:00:00'), ('RequiredDate', '1997-08-01T00:00:00'), ('ShipInfo', OrderedDict([('@ShippedDate', '1997-07-14T00:00:00'), ('ShipVia', '2'), ('Freight', '4.42'), ('ShipName', 'Great Lakes Food Market'), ('ShipAddress', '2732 Baker Blvd.'), ('ShipCity', 'Eugene'), ('ShipRegion', 'OR'), ('ShipPostalCode', '97403'), ('ShipCountry', 'USA')]))]), OrderedDict([('CustomerID', 'GRE

In [67]:
table_schema = {
    'fields': [
        {'name' : 'CustomerID', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name' : 'EmployeeID', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name' : 'OrderDate', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name' : 'RequiredDate', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name' : 'ShipInfo', 'type': 'RECORD', 'mode': 'NULLABLE', 'fields': [
            {'name' : 'ShipVia', 'type': 'STRING', 'mode': 'NULLABLE'},
            {'name' : 'Freight', 'type': 'STRING', 'mode': 'NULLABLE'},
            {'name' : 'ShipName', 'type': 'STRING', 'mode': 'NULLABLE'},
            {'name' : 'ShipAddress', 'type': 'STRING', 'mode': 'NULLABLE'},
            {'name' : 'ShipCity', 'type': 'STRING', 'mode': 'NULLABLE'},
            {'name' : 'ShipRegion', 'type': 'STRING', 'mode': 'NULLABLE'},
            {'name' : 'ShipPostalCode', 'type': 'STRING', 'mode': 'NULLABLE'},
            {'name' : 'ShipCountry', 'type': 'STRING', 'mode': 'NULLABLE'},
            {'name' : 'ShippedDate', 'type': 'STRING', 'mode': 'NULLABLE'},
        ]},
    ]
}

In [86]:
def cleanup(x):
    import copy
    y = copy.deepcopy(x)
    #print(x)
    if '@ShippedDate' in x['ShipInfo']:
        #print("#######")# optional attribute
        y['ShipInfo']['ShippedDate'] = x['ShipInfo']['@ShippedDate']
        del y['ShipInfo']['@ShippedDate']
    #print(y)
    return y

In [87]:
def get_orders(doc):
    #print(doc)
    for order in doc['Root']['Orders']['Order']:
        yield cleanup(order)

In [88]:
a=get_orders(document)
for i in a:
    print(i)

OrderedDict([('CustomerID', 'GREAL'), ('EmployeeID', '6'), ('OrderDate', '1997-05-06T00:00:00'), ('RequiredDate', '1997-05-20T00:00:00'), ('ShipInfo', OrderedDict([('ShipVia', '2'), ('Freight', '3.35'), ('ShipName', 'Great Lakes Food Market'), ('ShipAddress', '2732 Baker Blvd.'), ('ShipCity', 'Eugene'), ('ShipRegion', 'OR'), ('ShipPostalCode', '97403'), ('ShipCountry', 'USA'), ('ShippedDate', '1997-05-09T00:00:00')]))])
OrderedDict([('CustomerID', 'GREAL'), ('EmployeeID', '8'), ('OrderDate', '1997-07-04T00:00:00'), ('RequiredDate', '1997-08-01T00:00:00'), ('ShipInfo', OrderedDict([('ShipVia', '2'), ('Freight', '4.42'), ('ShipName', 'Great Lakes Food Market'), ('ShipAddress', '2732 Baker Blvd.'), ('ShipCity', 'Eugene'), ('ShipRegion', 'OR'), ('ShipPostalCode', '97403'), ('ShipCountry', 'USA'), ('ShippedDate', '1997-07-14T00:00:00')]))])
OrderedDict([('CustomerID', 'GREAL'), ('EmployeeID', '1'), ('OrderDate', '1997-07-31T00:00:00'), ('RequiredDate', '1997-08-28T00:00:00'), ('ShipInfo', O

In [106]:
import argparse
argv = None
p = beam.Pipeline(InteractiveRunner())


def printrow(element):
    print(element)
    print("##")

output = (p | 'files' >> beam.Create(['orders.xml']) |
'parse' >> beam.Map(lambda filename: parse_into_dict(filename)) |
'orders' >> beam.FlatMap(lambda doc: get_orders(doc)) | beam.Map(print))
#beam.io.WriteToBigQuery("gcp-bq-2021:dataset1.table2",
#                                       schema=table_schema,
#                                       write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND, #WRITE_TRUNCATE
#                                       create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
#                       custom_gcs_temp_location="gs://gcp-bq-2021"))

p.run()
#ib.show(output)

OrderedDict([('CustomerID', 'GREAL'), ('EmployeeID', '6'), ('OrderDate', '1997-05-06T00:00:00'), ('RequiredDate', '1997-05-20T00:00:00'), ('ShipInfo', OrderedDict([('ShipVia', '2'), ('Freight', '3.35'), ('ShipName', 'Great Lakes Food Market'), ('ShipAddress', '2732 Baker Blvd.'), ('ShipCity', 'Eugene'), ('ShipRegion', 'OR'), ('ShipPostalCode', '97403'), ('ShipCountry', 'USA'), ('ShippedDate', '1997-05-09T00:00:00')]))])
OrderedDict([('CustomerID', 'GREAL'), ('EmployeeID', '8'), ('OrderDate', '1997-07-04T00:00:00'), ('RequiredDate', '1997-08-01T00:00:00'), ('ShipInfo', OrderedDict([('ShipVia', '2'), ('Freight', '4.42'), ('ShipName', 'Great Lakes Food Market'), ('ShipAddress', '2732 Baker Blvd.'), ('ShipCity', 'Eugene'), ('ShipRegion', 'OR'), ('ShipPostalCode', '97403'), ('ShipCountry', 'USA'), ('ShippedDate', '1997-07-14T00:00:00')]))])
OrderedDict([('CustomerID', 'GREAL'), ('EmployeeID', '1'), ('OrderDate', '1997-07-31T00:00:00'), ('RequiredDate', '1997-08-28T00:00:00'), ('ShipInfo', O

In [95]:
a=range(3)
print(type(a))

<class 'range'>


In [109]:
parser_outputs = ['my', 'list', 'of', 'tables']
a=p | "Start" >> beam.Create(["example row"]) | "Split" >> beam.ParDo(MySplitFn()).with_outputs(*parser_outputs)

NameError: name 'MySplitFn' is not defined